In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [2]:
client : OpenAI = OpenAI()

In [4]:
from openai.types.beta import Assistant

# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("Asghar ibraheem baloch modified.pdf", "rb"),
  purpose='assistants'
)

print(file)

FileObject(id='file-olpAewrn9pZTETOrxAdXCKMp', bytes=88917, created_at=1717399457, filename='Asghar ibraheem baloch modified.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [17]:
assistant: Assistant = client.beta.assistants.create(
  name="Student Support Assistant",
  instructions="You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia U. Khan.",
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "file_search"}],
)

In [22]:
vector_store = client.beta.vector_stores.create(
  name="CV Documentation",
  file_ids=[file.id]
)

In [23]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [24]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)

Thread(id='thread_f50Brhcyiu3gvbyMxx6eYCco', created_at=1717400910, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [26]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Provide all contact detail provided in the CV?"
)

In [27]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Pakistani. The user is the student of PIAIC."
)

In [28]:
run: Run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

print(run)

Run(id='run_Lcc9QOogFJYDEB2fJGuNySj6', assistant_id='asst_maooVPhGekt5PkdfEwK8M4jB', cancelled_at=None, completed_at=1717401117, created_at=1717401114, expires_at=None, failed_at=None, incomplete_details=None, instructions='Please address the user as Pakistani. The user is the student of PIAIC.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, response_format='auto', started_at=1717401114, status='completed', thread_id='thread_f50Brhcyiu3gvbyMxx6eYCco', tool_choice='auto', tools=[FileSearchTool(type='file_search')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=124, prompt_tokens=2255, total_tokens=2379), temperature=1.0, top_p=1.0, tool_resources={})


In [30]:
# from openai.resources.beta.threads.messages.messages import SyncCursorPage 

messages: list[message] = client.beta.threads.messages.list(
  thread_id=thread.id
)

for m in reversed(messages.data):
  print(m.role + ": " + m.content[0].text.value)

user: Provide all contact detail provided in the CV?
assistant: The contact details provided in the CV are as follows:

- Name: Asghar Ibraheem Baloch
- Address: House #299, Sector 4-C Nai-abadi Baldia Town, Karachi
- Contact Number: +92309-5080420
- Email: asgharibraheembaloch@gmail.com

These details are available in the document "Asghar ibraheem baloch modified.pdf" .


In [32]:
delet_file = client.files.delete(file.id)